<img align="center" src="images/course.png" width="800">

# 16720 (B)  3D Reconstruction - Assignment 5 - q3
    Instructor: Kris                          TAs: Arka, Jinkun, Rawal, Rohan, Sheng-Yu

In [1]:
# Helper functions for this assignment. DO NOT MODIFY!!!
"""
Helper functions.
"""

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy
import cv2
import nbimporter
from q1 import eightpoint, sevenpoint, camera2, _epipoles, calc_epi_error, toHomogenous, _singularize
from q2 import find_M2, plot_3D

def plot_3D_dual(P_before, P_after):  
    matplotlib.use('TkAgg')
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title("Blue: before; red: after")
    ax.scatter(P_before[:,0], P_before[:,1], P_before[:,2], c = 'blue')
    ax.scatter(P_after[:,0], P_after[:,1], P_after[:,2], c='red')
    while True:
        x, y = plt.ginput(1, mouse_stop=2)[0]
        plt.draw()



## Q3: Bundle Adjustment
Bundle Adjustment is commonly used as the last step of every feature-based 3D reconstruction algorithm. Given a set of images depicting a number of 3D points from different viewpoints, bundle adjustment is the process of simultaneously refining the 3D coordinates along with the camera parameters. It minimizes reprojection error, which is the squared sum of distances between image points and predicted points. In this section, you will implement bundle adjustment algorithm by yourself. Specifically,


- In Q3.1, you need to implement a RANSAC algorithm to estimate the fundamental matrix F and all the inliers.
- In Q3.2, you will need to write code to parameterize Rotation matrix $\mathbf{R}$ using [Rodrigues formula](https://en.wikipedia.org/wiki/Rodrigues\%27\_formul) (Please check [this pdf](https://www2.cs.duke.edu/courses/fall13/compsci527/notes/rodrigues.pdf) for a detailed explanation), which will enable the joint optimization process for Bundle Adjustment.
- Q3.3, you will need to first write down the objective function in rodriguesResidual, and do the bundleAdjustment.

### Q3.1 RANSAC for Fundamental Matrix Recovery (15 pt implementation)

In some real world applications, manually determining correspondences is infeasible and often there will be noisy correspondences. Fortunately, the RANSAC method seen (and implemented in previous assignments) in class can be applied to the problem of fundamental matrix estimation.

Implement the above algorithm with the signature:
```
[F, inliers] = ransacF(pts1, pts2, M)
```

where `M` is defined in the same way as when we calculate the fundamental matrix and inliers is a boolean vector of size equivalent to the number of points. Here inliers are set to true only for the points that satisfy the threshold defined for the given fundamental matrix F.

We have provided some noisy coorespondances in some\_corresp\_noisy.npz in which around $75\%$ of the points are inliers. Compare the result of RANSAC with the result of the eight-point algorithm when ran on the noisy correspondences. 

**Hints:** Use the seven point to compute the fundamental matrix from the minimal set of points. Then compute the inliers, and refine your estimate using all the inliers.


In [2]:
def find_best_Fs(Fs_list, pts1_homo, pts2_homo):
    '''
    To find the best Fs from the list of Fs we get for each solution of the cubic equation
    '''
    ress = []
    F_res = []
    for Fs in Fs_list:
        res = calc_epi_error(pts1_homo, pts2_homo, Fs)
        F_res.append(Fs)
        ress.append(np.mean(res))
    min_idx = np.argmin(np.abs(np.array(ress)))
    Fs_best = F_res[min_idx]
    return Fs_best

def ransacF(pts1, pts2, M):
    '''
    Q3.1: RANSAC method.
        Input:  pts1, Nx2 Matrix
                pts2, Nx2 Matrix
                M, a scaler parameter
        Output: F, the fundamental matrix
                inlier_curr, Nx1 bool vector set to true for inliers
    ***
    Hints:
    (1) You can use the calc_epi_error from q1 with threshold to calcualte inliers. Tune the threshold based on 
        the results/expected number of inliners. You can also define your own metric. 
    (2) Use the seven point alogrithm to estimate the fundamental matrix as done in q1
    (3) Choose the resulting F that has the most number of inliers
    (4) You can increase the nIters to bigger/smaller values
        
    '''
    N = pts1.shape[0]
    pts1_homo, pts2_homo = toHomogenous(pts1), toHomogenous(pts2)
    threshold = 10
    max_iteration = 1000
    best_inlier = 0
    inlier_curr = None
    # ----- TODO -----
    # YOUR CODE HERE
    # RANSAC loop:
    for i in range(max_iteration):
        rand_col_idxs = np.random.choice(N, size=7)
        pts1_sampled = pts1[rand_col_idxs]
        pts2_sampled = pts2[rand_col_idxs]
        Fs_list = sevenpoint(pts1_sampled, pts2_sampled, M) # do seven point for 7 sampled points
        Fs = find_best_Fs(Fs_list, pts1_homo, pts2_homo)
        ress = calc_epi_error(pts1_homo, pts2_homo, Fs) # calculate error wrt all points
        inliers = np.where(ress<threshold, True, False)
        n_inliers = np.sum(inliers)
        if n_inliers>best_inlier:
            inlier_curr = inliers
            best_inlier = n_inliers

    best_idxs = np.where(inlier_curr==1)
    # refining estimate based on all inliers
    F = eightpoint(pts1[best_idxs], pts2[best_idxs], M)
    # raise NotImplementedError()
    return F, inlier_curr


np.random.seed(0)
np.set_printoptions(precision=4, suppress=1)
correspondence = np.load('data/some_corresp_noisy.npz') # Loading noisy correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
im1 = plt.imread('data/im1.png')
im2 = plt.imread('data/im2.png')
F, inliners = ransacF(pts1, pts2, M=np.max([*im1.shape, *im2.shape]))


Optimization terminated successfully.
         Current function value: 0.000107
         Iterations: 8
         Function evaluations: 881


In [3]:
# Full set of tests; you will get full points for coding if passing the following tests. 
try:
    assert(np.sum(inliners) > len(pts1) * 0.7)
    print('Test passed!')
except:
    raise AssertionError('Test failed')

Test passed!


### Q3.2 Rodrigues and Invsere Rodrigues (10 pt implementation)
So far we have independently solved for the camera matrix, $\mathbf{M}_j$ and 3D projections, $\textbf{w}_i$. In bundle adjustment, we will jointly optimize the reprojection error with respect to the points $\textbf{w}_i$ and the camera matrix $\textbf{w}_j$.

$$
err = \sum_{ij} ||\textbf{x}_{ij} - Proj(\mathbf{C}_j, \textbf{w}_i)||^2,
$$
where $\textbf{w}_j = \mathbf{K}_j \mathbf{M}_j$.

For this homework, we are going to only look at optimizing the extrinsic matrix. The rotation matrix forms the Lie Group $\textbf{SO}(3)$ that doesn't satisfy the addition operation so it cannot be directly optimized. Instead, we parameterize the rotation matrix to axis angle using Rodrigues formula to the Lie Algebra $\mathfrak{so}(3)$, which is defined in $\mathbb{R}^3$. through which the least squares optimization process can be done to optimize the axis angle. Try to implement function

```
R = rodrigues(r)
```

as well as the inverse function that converts a rotation matrix $\mathbf{R}$ to a Rodrigues vector $\mathbf{r}$

```
r = invRodrigues(R)
```

Please refer to [Rodrigues formula](https://en.wikipedia.org/wiki/Rodrigues\%27\_formul)  and [this pdf](https://www2.cs.duke.edu/courses/fall13/compsci527/notes/rodrigues.pdf) for reference.


In [4]:
def rodrigues(r):
    '''
    Q3.2: Rodrigues formula.
        Input:  r, a 3x1 vector
        Output: R, a 3x3 rotation matrix
    '''
    # ----- TODO -----
    # YOUR CODE HERE
    # raise NotImplementedError()
    # from the formula on page 5 of the pdf
    theta = np.linalg.norm(r)
    u = r/theta # axis of rotation
    u = u.reshape((3,1))
    u_x = np.array([[0.0, -u.item(2), u.item(1)],    # skew symmetric form of vector u
                [u.item(2), 0.0, -u.item(0)],
                [-u.item(1), u.item(0), 0.0]])
    R = np.cos(theta)*np.identity(3) + (1-np.cos(theta))*(u@u.T) + np.sin(theta)*u_x

    return R

def SHalf(r):
    theta = np.linalg.norm(r)

    if theta == np.pi and ( (r[0] == r[1] and r[0] == 0 and r[1] == 0 and r[2] < 0) or  (r[0] == 0 and r[1] < 0) or (r[0] < 0) ):
        return -r

    return r

def arctan2(y, x):
    if x > 0:
        return np.arctan(y/x)
    elif x<0:
        return np.pi + np.arctan(y/x)
    elif x == 0 and y < 0:
        return -np.pi/2
    elif x ==0 and y > 0:
        return np.pi/2

def invRodrigues(R):
    '''
    Q5.2: Inverse Rodrigues formula.
        Input:  R, a 3x3 rotation matrix
        Output: r, a 3x1 vector
    '''
    # ----- TODO -----
    # YOUR CODE HERE
    # raise NotImplementedError()
    # again from pg5 of pdf
    A = (R-R.T)/2
    rho = np.array([A[2,1], A[0,2], A[1,0]])
    s = np.linalg.norm(rho)
    c = (R[0,0]+R[1,1]+R[2,2]-1)/2

    if s==0 and c==1:
        r = np.zeros((3,))
    elif s==0 and c==-1:
        R_ = R + np.identity(3)
        cols_R_ = [R_[:,0], R_[:,1], R_[:,2]]
        for col in cols_R_:
            if np.any(col!=0):
                v = col
                u = v/np.linalg.norm(v)
                r_ = u*np.pi
                r = SHalf(r_)
                break
    elif s!=0:
        u = rho/s
        theta = arctan2(s,c)
        r = u*theta
    
    return r
    

In [5]:
# Simple Tests to verify your implmentation:
from scipy.spatial.transform import Rotation as sRot
rotVec = sRot.random()
mat = rodrigues(rotVec.as_rotvec())

try:
    assert(np.linalg.norm(rotVec.as_rotvec() - invRodrigues(mat)) < 1e-3)
    assert(np.linalg.norm(rotVec.as_matrix() - mat) < 1e-3)
    print('Test passed!') 
except:
    print('Test failed!')


Test passed!


In [6]:
# Hidden Tests

### Q3.3 Bundle Adjustment (10 pt writeup)

In this section, you need to implement the bundle adjustment algorithm. Using the parameterization you implemented in the last question, write an objective function for the extrinsic optimization:

```
residuals = rodriguesResidual(K1, M1, p1, K2, p2, x)
```
where x is the flattened concatenation of $\mathbf{w}$, $\mathbf{r}_2$, and $\mathbf{t}_2$.
$\mathbf{w}$ are the 3D points; $\mathbf{r}_2$ and $\mathbf{t}_2$ are the rotation (in the Rodrigues vector form) and translation vectors associated with the projection matrix $\mathbf{M}_2$; $p1$ and $p2$ are 2D coordinates of points in image 1 and 2, respectively. The `residuals` are the difference between the original image projections and the estimated projections (the square of $2$-norm of this vector corresponds to the error we computed in Q3.2):
```
residuals = numpy.concatenate([(p1-p1').reshape([-1]), (p2-p2').reshape([-1])])
```

Use this objective function and Scipy's nonlinear least squares optimizer $\texttt{leastsq}$ write a function to optimize for the best extrinsic matrix and 3D points using the inlier correspondences from some_corresp_noisy.npz and the RANSAC estimate of the extrinsics and 3D points as an initialization.

```
[M2, w, o1, o2] = bundleAdjustment(K1, M1, p1, K2, M2_init, p2, p_init)
```

Try to extract the rotation and translation from `M2_init`, then use `invRodrigues` you implemented previously to transform the rotation, concatenate it with translation and the 3D points, then the concatenate vector are variables to be optimized. After obtaining optimized vector, decompose it back to rotation using `Rodrigues` you implemented previously, translation and 3D points coordinates.

<span style='color:red'>**Output:**</span> In your write-up: include an image of output of the `plot_3D_dual` function by passing in the original 3D points and the optimized points. Also include the before and after reprojection error for the `rodriguesResidual` function.



In [7]:
def rodriguesResidual(K1, M1, p1, K2, p2, x):
    '''
    Q3.3: Rodrigues residual.
        Input:  K1, the intrinsics of camera 1
                M1, the extrinsics of camera 1
                p1, the 2D coordinates of points in image 1
                K2, the intrinsics of camera 2
                p2, the 2D coordinates of points in image 2
                x, the flattened concatenationg of P, r2, and t2.
        Output: residuals, 4N x 1 vector, the difference between original 
                and estimated projections
    '''
    residuals = None
    # ----- TODO -----
    # YOUR CODE HERE
    # raise NotImplementedError()
    n = p1.shape[0]
    P = x[:3*n].reshape((n,3))
    r = x[3*n:3*n+3]
    t = x[3*n+3:3*n+6, None]

    R = rodrigues(r)
    M2 =  np.hstack( [R, t] )
    C1 = K1.dot(M1)
    C2 = K2.dot(M2)
    
    P_homo = np.vstack( [P.T, np.ones(n) ] )

    p1__homo = C1 @ P_homo
    p2__homo = C2 @ P_homo

    p1_ = p1__homo.T
    p2_ = p2__homo.T

    # normalize
    p1_ = (p1_.T/p1_[:,-1]).T
    p2_ = (p2_.T/p2_[:,-1]).T

    # non-homogenize
    p1_ = p1_[:, :-1]
    p2_ = p2_[:, :-1]

    residuals = np.concatenate([(p1-p1_).reshape([-1]), (p2-p2_).reshape([-1])])

    return residuals



def bundleAdjustment(K1, M1, p1, K2, M2_init, p2, P_init):
    '''
    Q3.3 Bundle adjustment.
        Input:  K1, the intrinsics of camera 1
                M1, the extrinsics of camera 1
                p1, the 2D coordinates of points in image 1
                K2,  the intrinsics of camera 2
                M2_init, the initial extrinsics of camera 1
                p2, the 2D coordinates of points in image 2
                P_init, the initial 3D coordinates of points
        Output: M2, the optimized extrinsics of camera 1
                P2, the optimized 3D coordinates of points
                o1, the starting objective function value with the initial input
                o2, the ending objective function value after bundle adjustment
    
    ***
    Hints:
    (1) Use the scipy.optimize.minimize function to minimize the objective function, rodriguesResidual. 
        You can try different (method='..') in scipy.optimize.minimize for best results. 
    '''
    obj_start = obj_end = 0
    # ----- TODO -----
    # YOUR CODE HERE

    R_init = M2_init[:, 0:3]
    r_init = invRodrigues(R_init)
    t_init = M2_init[:, 3].flatten()
    x_init = np.concatenate([P_init.flatten(), r_init, t_init])

    # defining the objective function
    func = lambda x: (rodriguesResidual(K1, M1, p1, K2, p2, x)** 2).sum()
    # initial objective function value
    obj_start = func(x_init)
    print('Optimization started')
    res = scipy.optimize.minimize(func, x_init)
    print('Optimization completed')
    x_opt = res.x
    obj_end = func(x_opt)

    n = p1.shape[0]
    P_opt = x_opt[:3*n].reshape((n,3))
    r_opt = x_opt[3*n:3*n+3]
    t_opt = x_opt[3*n+3:3*n+6, None]
    R_opt = rodrigues(r_opt)
    M2_opt =  np.hstack([R_opt, t_opt])

    P = P_opt
    M2 = M2_opt

    return M2, P, obj_start, obj_end


In [8]:
# Visualization:
np.random.seed(0)
correspondence = np.load('data/some_corresp_noisy.npz') # Loading noisy correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
im1 = plt.imread('data/im1.png')
im2 = plt.imread('data/im2.png')

M=np.max([*im1.shape, *im2.shape])
F, inliners = ransacF(pts1, pts2, M)
pts1_inliners = pts1[inliners.squeeze(), :]
pts2_inliners = pts2[inliners.squeeze(), :]

M1 = np.hstack((np.identity(3), np.zeros(3)[:,np.newaxis]))
C1 = K1.dot(M1)
M2_init,C2, P_init = find_M2(F, pts1_inliners, pts2_inliners, intrinsics)

M2, P_final, obj_start, obj_end = bundleAdjustment(K1, M1, pts1_inliners, K2, M2_init, pts2_inliners, P_init)
print(f"Before {obj_start}, After {obj_end}")
# plot_3D_dual(P_init, P_final) ## comment to visualize, uncomment when submitting to gradescope


Optimization terminated successfully.
         Current function value: 0.000107
         Iterations: 8
         Function evaluations: 881
[[    0.     -1525.9      -87.87       0.    ]
 [ 1520.4        0.       108.32       0.    ]
 [  -27.1754 -1527.1232   -64.4339  1518.5012]
 [-1520.3281    38.0712  -103.5382   -30.8262]]
[[    0.     -1525.9      -87.87       0.    ]
 [ 1520.4        0.       108.32       0.    ]
 [  -27.1754 -1527.1232   -64.4339 -1518.5012]
 [-1520.3281    38.0712  -103.5382    30.8262]]
[[    0.     -1525.9      -87.87       0.    ]
 [ 1520.4        0.       108.32       0.    ]
 [   51.2007 -1448.9265  -484.7536  1518.5012]
 [ 1519.8403    22.3438   114.6869   -30.8262]]
[[    0.     -1525.9      -87.87       0.    ]
 [ 1520.4        0.       108.32       0.    ]
 [   51.2007 -1448.9265  -484.7536 -1518.5012]
 [ 1519.8403    22.3438   114.6869    30.8262]]
Best Error 352.84188176481746
Optimization started
Optimization completed
Before 352.84188176482445, After

In [9]:
plot_3D_dual(P_init, P_final)

TclError: invalid command name "pyimage50"